# Creating and processing the dataset

In [ ]:
# Creating the dataset
from src.data.make_dataset import generate_synthetic_data, DataRequestInput
from datetime import date

start_date = date(2015, 1, 1)  # Year, Month, Day
end_date = date(2023, 12, 31)  # Year, Month, Day
data_request = DataRequestInput(start_date=start_date, end_date=end_date)
synthetic_data = generate_synthetic_data(data_request)

In [ ]:
# Preprocessing the dataset
from src.data.preprocess import preprocess_data, PreprocessConfig

preprocess_config = PreprocessConfig()  # using the default set values!
train_data, val_data, test_data = preprocess_data(config=preprocess_config)

# Creating the environment And training the agent

In [ ]:
from src.training.Model_TrainTest import Model_train, ModelEvaluationConfig

actions_list = {
    "main_turbine_output": [0, 500],
    "secondary_turbine_output": [0, 500],
    "current_fuel_mixture_ratio": [0, 1],
    "generator_excitation": [0, 1],
    "emissions_control_intensity": [0, 1],
}

internal_states_initialValue = {
    "current_power_output": 0,
    "fuel_consumption_rate": 0,
    "emissions_levels": 0,
    "current_operating_costs": 0,
    "emissions_quota": 40000,
    "hours_main_turbine_since_maintenance": 0,
    "hours_secondary_turbine_since_maintenance": 0,
    "current_fuel_mixture_ratio": 0.5,
    "Initial_storage": 0,
}

environment_variables = {
    "Energy_storage_capacity": 15000,
    "Turbine_maintenance_time": 10,
    "Turbine_use_b4_maintenance": 100,
}

Model_config = ModelEvaluationConfig(
    agent_possible_actions=actions_list,
    train_total_timesteps=350000,
    internalStates_InitialVal=internal_states_initialValue,
    environment_variables=environment_variables,
    test_timesteps=1000,
)

TheMatrix = Model_train(Model_config)

In [ ]:
# training the agent
train_environment, val_environment, agent, Reward, accumulative_reward = (
    TheMatrix.train_agent()
)

In [ ]:
import matplotlib.pyplot as plt


def visualize_results(rewards):
    plt.figure(figsize=(10, 5))
    plt.plot(rewards)
    plt.title("Cumulative Reward over Time")
    plt.xlabel("Steps")
    plt.ylabel("Cumulative Reward")
    plt.show()


visualize_results(accumulative_reward)

# Testing

In [ ]:
# from src.training.Model_TrainTest import Model_train, ModelEvaluationConfig


# Model_config =ModelEvaluationConfig(
#     agent_possible_actions=actions_list,
#     train_total_timesteps=500000,
#     internalStates_InitialVal = internal_states_initialValue,
#     environment_variables=environment_variables,
#     test_timesteps=2000,
# )

# TheMatrix = Model_train(Model_config)

test_accum_rewards, test_rewards, test_observation_space = TheMatrix.test_agent()

In [ ]:
def visualize_results(rewards):
    plt.figure(figsize=(10, 5))
    plt.plot(rewards)
    plt.title("Cumulative Reward over Time")
    plt.xlabel("Steps")
    plt.ylabel("Cumulative Reward")
    plt.show()


visualize_results(test_rewards)

In [ ]:
import numpy as np

internal_state_variable = []
variable_index = 18

for jj in range(len(test_observation_space)):
    step_observation = test_observation_space[jj]
    # print(step_observation.shape)
    internal_state_variable.append(step_observation[0, variable_index])
internal_state_variable = np.array(internal_state_variable)

In [ ]:
def visualize_results(rewards):
    plt.figure(figsize=(10, 5))
    plt.plot(rewards)
    plt.title("Cumulative Reward over Time")
    plt.xlabel("Steps")
    plt.ylabel("Cumulative Reward")
    plt.show()


visualize_results(internal_state_variable)

In [ ]:
# SAVING RESULTS
# INTERNAL STATES
import numpy as np
import pandas as pd

internal_states = []
for jj in range(len(test_observation_space)):
    observation_step = test_observation_space[jj]
    internal_states.append(observation_step[0, :])

variable_id_list = [
    f"Internal_variable_{ii}" for ii in range(len(observation_step[0, :]))
]
internal_states_pd = pd.DataFrame(internal_states, columns=variable_id_list)

# REWARDS AND ACCUMMULATIVE REWARDS
combined_rewards = np.transpose(np.array([test_rewards, test_accum_rewards]))
rewards_pd = pd.DataFrame(combined_rewards, columns=["rewards", "accum_rewards"])

complete_data = pd.concat([internal_states_pd, rewards_pd], axis=1)

file_name = "PPOAgent_350000Steps_results.csv"
complete_data.to_csv(file_name)

# Article plots

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go


file_path = "data/processed/train.csv"
df = pd.read_csv(file_path)

columns_to_normalize = df.columns[df.columns.get_loc("index") + 1 :]
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

df = df.iloc[:50, :]

fig = go.Figure()

for col in columns_to_normalize:
    fig.add_trace(go.Scatter(x=df["index"], y=df[col], mode="lines", name=col))

# Update layout
fig.update_layout(
    title="Normalised Data  as a function of time",
    xaxis_title="time (hr)",
    yaxis_title="Normalized Value",
    width=1000,
    height=500,
)

fig.show()

In [ ]:
# ACCUMMULATIVE REWARD
import plotly.graph_objects as go

trained_Agent = pd.read_csv("PPOAgent_350000Steps_results.csv")
initial_Agent = pd.read_csv("PPOAgent_10000Steps_results.csv")

# parameter to plot
parameter = "accum_rewards"

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=trained_Agent.index,
        y=trained_Agent[parameter],
        mode="lines",
        name="500k steps trained",
    )
)

fig.add_trace(
    go.Scatter(
        x=trained_Agent.index,
        y=initial_Agent[parameter],
        mode="lines",
        name="10k steps trained",
    )
)

fig.update_layout(
    yaxis_title="Accummulated reward",
    xaxis_title="time (hrs)",
    width=800,
    height=600,
    # yaxis=dict(range=[-1e8, 1e7]),
    # xaxis=dict(range=[0, 150])
    # yaxis_type = "log"
)
fig.show()

In [ ]:
# OPERATIONAL PARAMETERS

# parameter to plot
parameter = "Internal_variable_18"

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=trained_Agent.index,
        y=trained_Agent[parameter],
        mode="lines",
        name="500k steps trained",
    )
)

fig.add_trace(
    go.Scatter(
        x=trained_Agent.index,
        y=initial_Agent[parameter],
        mode="lines",
        name="10k steps trained",
    )
)

fig.update_layout(
    yaxis_title="Accummulated reward",
    xaxis_title="time (hrs)",
    width=800,
    height=600,
    # yaxis_type = "log"
)
fig.show()